---
## Training

**Training the deep learning model for mask detection on faces**

In [1]:
from keras.preprocessing import image
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D

Using TensorFlow backend.


---
### Preparing input data

**Dataset used**

https://www.kaggle.com/omkargurav/face-mask-dataset

**Data directory structure**

The dataset directory is re-structured in the following format:

* data  
  * train
    * 1_without_mask -> contains 2828 images
    * 2_with_mask -> contains 2725 images
  * test
    * 1_without_mask -> contains 1000 images
    * 2_with_mask -> contains 1000 images

**Steps**

* The images are resized and are fed into the deep learning model in batches
* Data Augmentation: Images are rotated, cropped, zoomed, flipped etc. randomly
* _train_set_gen_ is the data flow for training the model
* _test_set_gen_ is the data flow for validating the model

In [2]:
IMAGE_SIZE = (128,128,3)
BATCH_SIZE = 32

In [3]:
im_generator = image.ImageDataGenerator(rotation_range=30,
                                 width_shift_range=0.15,
                                 height_shift_range=0.15,
                                 shear_range=0.1,
                                 zoom_range=0.1,
                                 horizontal_flip=True,
                                 fill_mode='nearest',
                                 rescale=1/255)

In [4]:
train_set_gen = im_generator.flow_from_directory('data/train',
                                               target_size=IMAGE_SIZE[:2],
                                               batch_size=BATCH_SIZE,
                                               class_mode='binary')

test_set_gen = im_generator.flow_from_directory('data/test',
                                                target_size=IMAGE_SIZE[:2],
                                                batch_size=BATCH_SIZE,
                                                class_mode='binary')

Found 5553 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


#### Class indices

without_mask corresponds to class 0, with_mask corresponds to class 1

In [5]:
test_set_gen.class_indices

{'1_without_mask': 0, '2_with_mask': 1}

---
### Building and training the model

The mask detection model's output is a single number between 0 and 1 representing _probability of a face being classified as 'with mask'_

In [6]:
model = Sequential()

model.add(Conv2D(filters=8, kernel_size=(3,3), input_shape=IMAGE_SIZE, activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters=32, kernel_size=(3,3), dilation_rate=(2,2), input_shape=IMAGE_SIZE, activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters=64, kernel_size=(3,3), dilation_rate=(2,2), input_shape=IMAGE_SIZE, activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())

model.add(Dense(128))
model.add(Activation('relu'))

model.add(Dropout(0.5))

# Last layer, it's binary, 0=without_mask , 1=with_mask
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 126, 126, 8)       224       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 63, 63, 8)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 59, 59, 32)        2336      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 29, 29, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 25, 25, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)             

In [7]:
results = model.fit_generator(train_set_gen,
                              epochs=20,
                              steps_per_epoch=100,
                              validation_data=test_set_gen,
                              validation_steps=30)

Epoch 1/20
100/100 [==============================] - 71s 713ms/step - loss: 0.5040 - accuracy: 0.7594 - val_loss: 0.5405 - val_accuracy: 0.8104
Epoch 2/20
 99/100 [============================>.] - ETA: 0s - loss: 0.3499 - accuracy: 0.8497

C:\Users\Dharmveer\Anaconda3\envs\computer_vision\lib\site-packages\PIL\Image.py:952: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


100/100 [==============================] - 71s 714ms/step - loss: 0.3498 - accuracy: 0.8499 - val_loss: 0.4800 - val_accuracy: 0.7135
Epoch 3/20
100/100 [==============================] - 66s 663ms/step - loss: 0.2996 - accuracy: 0.8747 - val_loss: 0.6205 - val_accuracy: 0.8326
Epoch 4/20
100/100 [==============================] - 66s 658ms/step - loss: 0.2566 - accuracy: 0.9016 - val_loss: 0.3472 - val_accuracy: 0.8000
Epoch 5/20
100/100 [==============================] - 69s 690ms/step - loss: 0.2411 - accuracy: 0.9083 - val_loss: 0.3709 - val_accuracy: 0.7850
Epoch 6/20
100/100 [==============================] - 70s 702ms/step - loss: 0.2284 - accuracy: 0.9134 - val_loss: 0.4014 - val_accuracy: 0.8354
Epoch 7/20
100/100 [==============================] - 78s 778ms/step - loss: 0.2181 - accuracy: 0.9177 - val_loss: 0.2567 - val_accuracy: 0.8422
Epoch 8/20
100/100 [==============================] - 69s 686ms/step - loss: 0.2225 - accuracy: 0.9159 - val_loss: 0.2257 - val_accuracy: 0.8

### Saving the model

In [8]:
model.save("mask_cnn.h5")

---